In [1]:
library(data.table)
library(tidyverse)
library(lme4)
library(performance)
library(cowplot)
library(emmeans)
library(marginaleffects)
library(pbapply)

── Attaching core tidyverse packages ────────────────────────────────────────────────────────────────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::between()     masks data.table::between()
✖ dplyr::filter()      masks stats::filter()
✖ dplyr::first()       masks data.table::first()
✖ lubridate::hour()    masks data.table::hour()
✖ lubridate::isoweek() masks data.table::isoweek()
✖ dplyr::lag()         masks stats::lag()
✖ dplyr::last()        masks data.table::last()
✖ lubridate::mday()    masks data.table::mday()
✖ lubridate::minute()  masks data.table::minute()
✖ lubridate::month()   masks data.table::month()
✖ lubridate::quarter() masks data.table::quarter()
✖ lubri

In [2]:
dt <- fread("../../natera_recomb/analysis/co_post_process/results/v30b_heuristic_90_nsib_qual.crossover_filt.deCode_haldorsson19.hotspot_occupy.maternal.euploid.csv.gz") %>%
  .[, is_aneuploid_embryo := FALSE]

dt2 <- fread("../../natera_recomb/analysis/co_post_process/results/v30b_heuristic_90_nsib_qual.crossover_filt.deCode_haldorsson19.hotspot_occupy.maternal.aneuploid.csv.gz") %>%
  .[, is_aneuploid_embryo := TRUE]

dt <- rbind(dt, dt2)

# dt[, AVGSIGMA := weighted.mean(sigma_baf, cM_len), by = uid]
# dt[, AVGPI0 := weighted.mean(pi0_baf, cM_len), by = uid]

# dt[egg_donor == 1, patient_age := as.numeric(25)]

# This just kind of filters out the zeros and 1s?
df <- dt
# df <- dt %>% filter((mean_alpha_mat > 1e-4)& (mean_alpha_mat < 0.999) )
df <- df %>% filter(nchrom >= 17)
head(df)

uid,lower_95_alpha_mat,mean_alpha_mat,upper_95_alpha_mat,nco_pass_mat,nchrom,mother,father,child,euploid,⋯,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,is_aneuploid_embryo
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
3999947049_R04C02+3999947049_R03C02+10013440035_R03C02,0.2508366,0.43812721,0.6153844,48,18,3999947049_R04C02,3999947049_R03C02,10013440035_R03C02,TRUE,⋯,-0.001504400,-0.003202090,-0.000751128,0.00234529,0.002164140,0.005414180,0.000328795,-0.00324049,0.001168260,FALSE
9402204156_R04C01+9402204156_R05C01+9997249137_R02C02,0.1973250,0.43812721,0.6688960,25,17,9402204156_R04C01,9402204156_R05C01,9997249137_R02C02,TRUE,⋯,-0.001095620,0.002572000,-0.002592810,0.00160076,0.000139387,0.005994390,0.000235390,-0.00109851,0.002663930,FALSE
202701740047_R05C01+202701740047_R06C01+202701740034_R05C01,0.1204021,0.36120429,0.6053510,29,17,202701740047_R05C01,202701740047_R06C01,202701740034_R05C01,TRUE,⋯,-0.000479670,0.003296430,0.006107020,-0.00179910,-0.001394820,0.000461605,-0.001074730,0.00866010,0.000778493,FALSE
200693600062_R01C02+200693600062_R02C02+202010620100_R04C01,0.1337800,0.41806036,0.6822739,22,17,200693600062_R01C02,200693600062_R02C02,202010620100_R04C01,TRUE,⋯,0.000632574,0.003123270,-0.004525200,0.00152565,0.000489531,0.002093700,-0.001285610,0.01224150,0.000634308,FALSE
9446208058_R05C01+9446208058_R06C01+9992577049_R05C01,0.0668905,0.28762584,0.5217391,31,17,9446208058_R05C01,9446208058_R06C01,9992577049_R05C01,TRUE,⋯,0.005335890,-0.003744260,0.002361340,-0.00126484,0.001746850,-0.003679830,-0.004563660,-0.00530411,-0.001815610,FALSE
203962720020_R03C02+203962720020_R04C02+204568340046_R06C01,0.0000010,0.02341232,0.2140474,33,17,203962720020_R03C02,203962720020_R04C02,204568340046_R06C01,TRUE,⋯,0.000268920,-0.000330688,-0.003791650,0.00484152,-0.009473260,-0.002773430,-0.001690740,-0.00332615,-0.000658282,FALSE


In [3]:
tail(df)

uid,lower_95_alpha_mat,mean_alpha_mat,upper_95_alpha_mat,nco_pass_mat,nchrom,mother,father,child,euploid,⋯,PC12,PC13,PC14,PC15,PC16,PC17,PC18,PC19,PC20,is_aneuploid_embryo
<chr>,<dbl>,<dbl>,<dbl>,<int>,<int>,<chr>,<chr>,<chr>,<lgl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
3998633005_R02C01+3998633005_R01C01+100954520028_R06C02,0.08026840,0.2374587,0.4180604,44,17,3998633005_R02C01,3998633005_R01C01,100954520028_R06C02,FALSE,⋯,-0.004949370,0.000929288,-0.001464710,-0.008221480,-0.004456550,0.000401423,0.002364590,-0.005465880,-4.33118e-03,TRUE
101242990021_R04C01+101034430083_R03C01+200151560160_R03C01,0.13043552,0.3010037,0.4949833,37,17,101242990021_R04C01,101034430083_R03C01,200151560160_R03C01,FALSE,⋯,0.002070450,-0.002240010,-0.002090190,0.001140000,0.002727360,0.006968590,0.003143070,-0.000807731,-1.74894e-05,TRUE
201672610194_R01C01+201672610194_R02C01+204572560004_R01C02,0.14715790,0.3979935,0.6454847,25,17,201672610194_R01C01,201672610194_R02C01,204572560004_R01C02,FALSE,⋯,-0.008397750,-0.015521900,0.004713210,-0.000878522,0.001303050,0.005902350,0.000142281,-0.000459776,3.68263e-03,TRUE
203447090061_R01C01+203447090055_R06C02+203488000017_R04C02,0.04347917,0.2073584,0.4013380,43,17,203447090061_R01C01,203447090055_R06C02,203488000017_R04C02,FALSE,⋯,0.001117340,-0.000974639,0.000235214,-0.000194616,0.001323850,0.004921680,-0.002111820,-0.001211640,-4.47393e-03,TRUE
203096620121_R04C01+203096620121_R03C01+203124470025_R03C02,0.00000100,0.0936463,0.2742479,45,18,203096620121_R04C01,203096620121_R03C01,203124470025_R03C02,FALSE,⋯,0.000775748,-0.000250537,0.000822514,0.001220700,0.000631824,0.001921380,0.000272966,0.001097250,-8.55830e-04,TRUE
203791440028_R05C01+203791440044_R06C01+204221260024_R02C02,0.06354602,0.2976593,0.5585283,22,18,203791440028_R05C01,203791440044_R06C01,204221260024_R02C02,FALSE,⋯,-0.002718150,-0.000152026,-0.001017450,-0.003533110,0.003341910,0.005834630,-0.000393231,-0.003929130,-8.97747e-04,TRUE


In [4]:
# Compute the total groupwide mean 
df %>% group_by(maternal_meiotic_aneuploidy) %>% summarize(avg=mean(mean_alpha_mat), median=median(mean_alpha_mat), n=n(), sd=sd(mean_alpha_mat), se=sd/sqrt(n))

maternal_meiotic_aneuploidy,avg,median,n,sd,se
<lgl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>
FALSE,0.2404669,0.2374587,547,0.1288203,0.005507957
TRUE,0.2232527,0.2107029,105,0.1314487,0.012828084


In [5]:
# fit model
m1 <- glmer(data = df, 
            formula = mean_alpha_mat ~ (1 | IID) + scale(nco_pass_mat) + scale(Age) + PC1 + PC2 + PC3 + PC4 + PC5 + 
              PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 +
              PC18 + PC19 + PC20 + nchrom + scale(AVGSIGMA) + scale(AVGPI0) + scale(NEMBRYO) + maternal_meiotic_aneuploidy,
            family=Gamma(link = "log"),
            nAGQ = 0,
            control = glmerControl(optimizer = "bobyqa"))
summary(m1)

boundary (singular) fit: see help('isSingular')


Correlation matrix not shown by default, as p = 28 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 0) [glmerMod]
 Family: Gamma  ( log )
Formula: mean_alpha_mat ~ (1 | IID) + scale(nco_pass_mat) + scale(Age) +  
    PC1 + PC2 + PC3 + PC4 + PC5 + PC6 + PC7 + PC8 + PC9 + PC10 +  
    PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 + PC18 + PC19 +  
    PC20 + nchrom + scale(AVGSIGMA) + scale(AVGPI0) + scale(NEMBRYO) +  
    maternal_meiotic_aneuploidy
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
  -549.8   -415.7    304.9   -609.8      616 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9563 -0.7462  0.0032  0.7275  3.5275 

Random effects:
 Groups   Name        Variance Std.Dev.
 IID      (Intercept) 0.0000   0.0000  
 Residual             0.2613   0.5112  
Number of obs: 646, groups:  IID, 535

Fixed effects:
                                  Estimate Std. Error t value Pr(>|z|)    
(Intercept)                

In [6]:
emmeans(m1, "maternal_meiotic_aneuploidy", type = "response")

 maternal_meiotic_aneuploidy response      SE  df asymp.LCL asymp.UCL
 FALSE                          0.231 0.00513 Inf     0.221     0.242
  TRUE                          0.225 0.01184 Inf     0.203     0.249

Confidence level used: 0.95 
Intervals are back-transformed from the log scale 

In [7]:
# fit model without nco and nchrom covariates ... 
m2 <- glmer(data = df, 
            formula = mean_alpha_mat ~ (1 | IID) + scale(Age) + PC1 + PC2 + PC3 + PC4 + PC5 + 
              PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 + PC14 + PC15 + PC16 + PC17 +
              PC18 + PC19 + PC20 + scale(AVGSIGMA) + scale(AVGPI0) + scale(NEMBRYO) + maternal_meiotic_aneuploidy,
            family=Gamma(link = "log"),
            nAGQ = 0,
            control = glmerControl(optimizer = "bobyqa"))
summary(m2)

boundary (singular) fit: see help('isSingular')


Correlation matrix not shown by default, as p = 26 > 12.
Use print(obj, correlation=TRUE)  or
    vcov(obj)        if you need it




Generalized linear mixed model fit by maximum likelihood (Adaptive
  Gauss-Hermite Quadrature, nAGQ = 0) [glmerMod]
 Family: Gamma  ( log )
Formula: mean_alpha_mat ~ (1 | IID) + scale(Age) + PC1 + PC2 + PC3 + PC4 +  
    PC5 + PC6 + PC7 + PC8 + PC9 + PC10 + PC11 + PC12 + PC13 +  
    PC14 + PC15 + PC16 + PC17 + PC18 + PC19 + PC20 + scale(AVGSIGMA) +  
    scale(AVGPI0) + scale(NEMBRYO) + maternal_meiotic_aneuploidy
   Data: df
Control: glmerControl(optimizer = "bobyqa")

     AIC      BIC   logLik deviance df.resid 
  -551.6   -426.4    303.8   -607.6      618 

Scaled residuals: 
    Min      1Q  Median      3Q     Max 
-1.9268 -0.7195  0.0151  0.6894  3.7864 

Random effects:
 Groups   Name        Variance Std.Dev.
 IID      (Intercept) 0.0000   0.000   
 Residual             0.2694   0.519   
Number of obs: 646, groups:  IID, 535

Fixed effects:
                                  Estimate Std. Error t value Pr(>|z|)    
(Intercept)                      -1.326675   0.030511 -43.482  <

In [8]:
emmeans(m2, "maternal_meiotic_aneuploidy", type = "response")

 maternal_meiotic_aneuploidy response      SE  df asymp.LCL asymp.UCL
 FALSE                          0.232 0.00521 Inf     0.222     0.242
  TRUE                          0.226 0.01201 Inf     0.203     0.250

Confidence level used: 0.95 
Intervals are back-transformed from the log scale 